# moses统计翻译系统实战

In [13]:
%%bash

# 安装 Moses
# http://www.statmt.org/moses/?n=Development.GetStarted

# 下载数据集
corpus="$PWD/corpus"
mkdir -p $corpus
cd $corpus 
wget http://www.statmt.org/wmt13/training-parallel-nc-v8.tgz
tar zxvf training-parallel-nc-v8.tgz

training/news-commentary-v8.cs-en.cs
training/news-commentary-v8.cs-en.en
training/news-commentary-v8.de-en.de
training/news-commentary-v8.de-en.en
training/news-commentary-v8.es-en.en
training/news-commentary-v8.es-en.es
training/news-commentary-v8.fr-en.en
training/news-commentary-v8.fr-en.fr
training/news-commentary-v8.ru-en.en
training/news-commentary-v8.ru-en.ru


In [3]:
%%bash
corpus="$PWD/corpus"
head -n 5 $corpus/training/news-commentary-v8.fr-en.en

SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.
Lately, with gold prices up more than 300% over the last decade, it is harder than ever.
Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.
Wouldn’t you know it?
Since their articles appeared, the price of gold has moved up still further. Gold prices even hit a record-high $1,300 recently.


In [4]:
%%bash
corpus="$PWD/corpus"
head -n 5 $corpus/training/news-commentary-v8.fr-en.fr

SAN FRANCISCO – Il n’a jamais été facile d’avoir une discussion rationnelle sur la valeur du métal jaune.
Et aujourd’hui, alors que le cours de l’or a augmenté de 300 pour cent au cours de la dernière décennie, c’est plus difficile que jamais.
En décembre dernier, mes collègues économistes Martin Feldstein et Nouriel Roubini ont chacun publié une tribune libre dans laquelle ils doutaient courageusement du marché haussier, soulignant de manière sensée les risques liés à l’or.
Mais devinez ce qui s’est passé ?
Depuis la parution de leurs articles, le cours de l’or a encore grimpé, pour atteindre récemment un plus haut historique de 1300 dollars l’once.


In [7]:
%%bash
corpus="$PWD/corpus"
wc -l $corpus/training/news-commentary-v8.fr-en.{fr,en}

  157168 /Users/jjhu/remoteshare/MT/corpus/training/news-commentary-v8.fr-en.fr
  157168 /Users/jjhu/remoteshare/MT/corpus/training/news-commentary-v8.fr-en.en
  314336 total


In [8]:
%%bash
# tokenization: 对句子进行符号化
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
$mosesdecoder/scripts/tokenizer/tokenizer.perl -l en \
   < $corpus/training/news-commentary-v8.fr-en.en \
   > $corpus/news-commentary-v8.fr-en.tok.en

$mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr \
   < $corpus/training/news-commentary-v8.fr-en.fr \
   > $corpus/news-commentary-v8.fr-en.tok.fr


Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1


In [2]:
%%bash 

# 训练 truecaser: 将句子第一个字母变成小写
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
$mosesdecoder/scripts/recaser/train-truecaser.perl \
     --model $corpus/truecase-model.en --corpus     \
     $corpus/news-commentary-v8.fr-en.tok.en
 $mosesdecoder/scripts/recaser/train-truecaser.perl \
     --model $corpus/truecase-model.fr --corpus     \
     $corpus/news-commentary-v8.fr-en.tok.fr

In [4]:
%%bash 

# 将句子第一个字母变成小写
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
 $mosesdecoder/scripts/recaser/truecase.perl \
   --model $corpus/truecase-model.en         \
   < $corpus/news-commentary-v8.fr-en.tok.en \
   > $corpus/news-commentary-v8.fr-en.true.en
 $mosesdecoder/scripts/recaser/truecase.perl \
   --model $corpus/truecase-model.fr         \
   < $corpus/news-commentary-v8.fr-en.tok.fr \
   > $corpus/news-commentary-v8.fr-en.true.fr

In [5]:
%%bash 

# 将双语语料库中句子单词个数多于80的句子去除
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
$mosesdecoder/scripts/training/clean-corpus-n.perl \
    $corpus/news-commentary-v8.fr-en.true fr en \
    $corpus/news-commentary-v8.fr-en.clean 1 80

clean-corpus.perl: processing /home/jjhu/MT/corpus/news-commentary-v8.fr-en.true.fr & .en to /home/jjhu/MT/corpus/news-commentary-v8.fr-en.clean, cutoff 1-80, ratio 9
..........(100000).....
Input sentences: 157168  Output sentences:  155362


In [6]:
%%bash

corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
# 训练语言模型
lm="$PWD/lm"
mkdir $lm
cd $lm
$mosesdecoder/bin/lmplz -o 3 < $corpus/news-commentary-v8.fr-en.true.en > $lm/news-commentary-v8.fr-en.arpa.en
$mosesdecoder/bin/build_binary $lm/news-commentary-v8.fr-en.arpa.en $lm/news-commentary-v8.fr-en.blm.en

=== 1/5 Counting and sorting n-grams ===
Reading /home/jjhu/MT/corpus/news-commentary-v8.fr-en.true.en
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 4066728 types 62719
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:752628 2:9359020032 3:17548163072
Statistics:
1 62719 D1=0.622525 D2=0.981811 D3+=1.36293
2 906706 D1=0.742501 D2=1.07915 D3+=1.38352
3 2389081 D1=0.835943 D2=1.1625 D3+=1.34981
Memory estimate for binary LM:
type    MB
probing 63 assuming -p 1.5
probing 68 assuming -r models -p 1.5
trie    25 without quantization
trie    14 assuming -q 8 -b 8 quantization 
trie    24 assuming -a 22 array pointer compression
trie    12 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:752628 2:14507296 3:47781620


In [7]:
%%bash 

corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
working="$PWD/working"
lm="$PWD/lm"
# 训练翻译模型
mkdir $working
cd $working
nohup nice $mosesdecoder/scripts/training/train-model.perl -root-dir train \
 -corpus $corpus/news-commentary-v8.fr-en.clean                             \
 -f fr -e en -alignment grow-diag-final-and -reordering msd-bidirectional-fe \
 -lm 0:3:$lm/news-commentary-v8.fr-en.blm.en:8                          \
 -external-bin-dir $mosesdecoder/tools >& training.out &

In [8]:
%%bash 
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
working="$PWD/working"
lm="$PWD/lm"

# 利用dev 数据调整模型参数
cd $corpus
wget http://www.statmt.org/wmt12/dev.tgz
tar zxvf dev.tgz

# 处理dev 数据， tokenization + truecase
$mosesdecoder/scripts/tokenizer/tokenizer.perl -l en \
   < $corpus/dev/news-test2008.en > $corpus/news-test2008.tok.en
$mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr \
   < $corpus/dev/news-test2008.fr > $corpus/news-test2008.tok.fr
$mosesdecoder/scripts/recaser/truecase.perl --model truecase-model.en \
   < $corpus/news-test2008.tok.en > $corpus/news-test2008.true.en
$mosesdecoder/scripts/recaser/truecase.perl --model truecase-model.fr \
   < $corpus/news-test2008.tok.fr > $corpus/news-test2008.true.fr

dev/
dev/newstest2009-src.fr.sgm
dev/news-test2008-src.hu.sgm
dev/newssyscomb2009-ref.cs.sgm
dev/newstest2009-ref.en.sgm
dev/newstest2009-ref.hu.sgm
dev/newstest2010.cs
dev/newssyscomb2009-ref.hu.sgm
dev/newstest2010-src.es.sgm
dev/newstest2011-src.es.sgm
dev/newstest2010-src.fr.sgm
dev/newssyscomb2009-src.fr.sgm
dev/newstest2011-ref.de.sgm
dev/news-test2008-ref.es.sgm
dev/newssyscomb2009-ref.de.sgm
dev/newstest2011.fr
dev/news-test2008.cs
dev/news-test2008-ref.fr.sgm
dev/newstest2009-ref.cz.sgm
dev/newstest2011-src.de.sgm
dev/newstest2011-ref.cs.sgm
dev/raw/
dev/raw/newstest2011-src.fr.raw.sgm
dev/raw/newstest2011-src.es.raw.sgm
dev/raw/newstest2011-src.cs.raw.sgm
dev/raw/newstest2011-ref.fr.raw.sgm
dev/raw/newstest2011-ref.es.raw.sgm
dev/raw/newstest2011-ref.en.raw.sgm
dev/raw/newstest2011-src.de.raw.sgm
dev/raw/newstest2011-src.en.raw.sgm
dev/raw/newstest2011-ref.de.raw.sgm
dev/raw/newstest2011-ref.cs.raw.sgm
dev/news-test2008-src.fr.sgm
dev/newssyscomb2009-src.it.sgm
dev/newstest20

wget: /home/jjhu/Software/anaconda2/envs/py36/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /home/jjhu/Software/anaconda2/envs/py36/lib/libuuid.so.1: no version information available (required by wget)
wget: /home/jjhu/Software/anaconda2/envs/py36/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-01-20 01:52:41--  http://www.statmt.org/wmt12/dev.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13260990 (13M) [application/x-gzip]
Saving to: ‘dev.tgz’

     0K .......... .......... .......... .......... ..........  0%  252K 51s
    50K .......... .......... .......... .......... ..........  0%  479K 39s
   100K .......... .......... .......... .......... ..........  1% 28.9M 26s
   150K .......... .......... .......... .......... ..........  1%  102M 19s
   200K .......... .

In [ ]:
%%bash 
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
working="$PWD/working"
lm="$PWD/lm"

# 调节参数
cd $working
nohup nice $mosesdecoder/scripts/training/mert-moses.pl \
  $corpus/news-test2008.true.fr $corpus/news-test2008.true.en \
  $mosesdecoder/bin/moses train/model/moses.ini --mertdir $mosesdecoder/bin/ \
  &> mert.out &

In [ ]:
%%bash
corpus="$PWD/corpus"
mosesdecoder="$PWD/mosesdecoder"
working="$PWD/working"
lm="$PWD/lm"

# 处理test 数据
cd $corpus
$mosesdecoder/scripts/tokenizer/tokenizer.perl -l en \
   < $corpus/dev/newstest2011.en > $corpus/newstest2011.tok.en
$mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr \
   < $corpus/dev/newstest2011.fr > $corpus/newstest2011.tok.fr
$mosesdecoder/scripts/recaser/truecase.perl --model $corpus/truecase-model.en \
   < $corpus/newstest2011.tok.en > $corpus/newstest2011.true.en
$mosesdecoder/scripts/recaser/truecase.perl --model $corpus/truecase-model.fr \
   < $corpus/newstest2011.tok.fr > $corpus/newstest2011.true.fr
   
# 翻译test 数据
nohup nice $mosesdecoder/bin/moses            \
   -f $working/filtered-newstest2011/moses.ini   \
   < $corpus/newstest2011.true.fr                \
   > $working/newstest2011.translated.en         \
   2> $working/newstest2011.out 

# 自动评估：BLEU
$mosesdecoder/scripts/generic/multi-bleu.perl \
   -lc $corpus/newstest2011.true.en              \
   < $working/newstest2011.translated.en

### 资料补充

* Moses相关介绍: http://www.52nlp.cn/moses-introduction
* Moses基本框架: http://www.52nlp.cn/moses-basic-framework
* Moses的一些新变化: http://www.52nlp.cn/moses%E7%9A%84%E4%B8%80%E4%BA%9B%E6%96%B0%E5%8F%98%E5%8C%96